# Machine Learning and Artificial Intelligence
## Stefano Brilli s249914 - homework 2

## 1 Analysis definition
In this homework we will use another tool for classification, called Support Vector Machine, in order to classify flowers according to their features. The dataset is presented in one of the next chapters.

### 1.1 Definition of hyperplane
In a p-dimensional space, a hyperplane is a flat affine subspace of dimension p-1 (the word affine indicates that the subspace need not pass through the origin).
In three dimensions, a hyperplane is a flat two-dimensional subspace, that is a plane.
In two dimensions it's a line. The mathematical definition of a hyperplane is quite simple. In two dimensione, a hyperplane is defined by the equation {{r'\beta_{0}+\beta_{1}X_{1}-X_{2}=0'}}

In [7]:
Math(r'\beta_{0}+\beta_{1}X_{1}-X_{2}=0')

<IPython.core.display.Math object>

### 1.2 SVM briefly explanation
SVM is a supervised algorithm used for classifying data belonging to two or more classes. The objective is to find the hyperplane that separates well data.
It means that for each point we want to assign it a label according to what side it belongs to. 

#### 1.2.1 Large Margin Separation
If our data can be perfectly separated using a hyperplane, then there exists an infinte number of such hyperplanes.
We have to decide which of these hyperplanes we want to use to separate data.
A natural choice is the <i>large margin separation</i> (also called <i>maximal margin separation</i>), which is the separating hyperplane so that the distance between two point groups is as large as possible.

#### 1.2.2 Soft Margin Separation
In order to get a greater robustness to individual observations and obtain a better classification of most of the training observations, we can fit the model by using a soft margin classifier.
The idea here is: rather than seeking the largest possible margin so that every observation is not only on the correct side of the hyperplane but also on the correct side of the margin, we instead allow some observations to be on the incorrect side of the margin, or even the incorrect side of the hyperplane. The margin is called soft because it can be violated by some of the training observations.

### 1.3 The dataset
Our analysis will be performed on the iris data, available at https://archive.ics.uci.edu/ml/datasets/iris or by importing the data sets library from sklearn, that contains many useful datasets.
The data set contains 3 classes and 50 instances for each one, where each class refers to a type of iris plant.
The classes are:
<table style="border: 1px solid black;">
    <tr>
        <th style="border: 1px solid black">Class 1</th>
        <th style="border: 1px solid black;">Class 2</th>
        <th style="border: 1px solid black;">Class 3</th>
    </tr>
    <tr>
        <td style="border: 1px solid black;">Iris-setosa</th>
        <td style="border: 1px solid black;">Iris-versicolor</th>
        <td style="border: 1px solid black;">Iris-virginica</th>
    </tr>
    <tr>
        <td style="border: 1px solid black;">[0]</th>
        <td style="border: 1px solid black;">[1]</th>
        <td style="border: 1px solid black;">[2]</th>
    </tr>
</table>

For simplicity we will use just two features of the dataset: <i>sepal length in cm</i> and <i>sepal width in cm</i>.

### 1.4 Linear and non-linear SVM
As we will see in next chapers it's not always possible to find a linear separation between data. Other times it's better to compute a non-linear separation to optimize our separation model. In this homework we will use both a linear kernel and non linear one. 

### 1.5 Tools used
As for the previous homework, we will use the Python language to complete the analysis and sklearn libraries (http://scikit-learn.org/stable/documentation.html).

In [2]:
import numpy as np
from sklearn.svm import SVC
from sklearn import datasets, svm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from tabulate import tabulate
from IPython.display import Math

In [1]:
# This function load the iris dataset in memory and gets just the first two dimensions
def loadData():
    iris = datasets.load_iris()
    X = iris.data[:, :2]
    y = iris.target
    return X, y